In [20]:
from pydantic import BaseModel, Field
from typing import List

import numpy as np
from PIL import Image

import base64
from io import BytesIO
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
import os 

load_dotenv()

True

### Utilização da multimodalidade dos LLMs

Aqui vamos explorar a habilidade que os LLMs possuem de lidar com múltiplos tipos de dados. Para isso, vamos utilizar o modelo `gpt4o`, que é um modelo da OpenAI com performance relativamente adequada para extração de informações de texto e imagens.

In [15]:
model_name = "gpt-4o-2024-08-06"

llm = ChatOpenAI(model=model_name, temperature=0.01)

def get_list_food(paths: List[str], 
                          provider="openai"):

    # Caminho para a imagem em Base64
    image_urls = paths 
    delta = len(image_urls)
    images_data = []
    
    for image_url in image_urls:
        with open(image_url, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
            images_data.append(image_data)
    
    system_prompt = """
    Você vai receber uma séria de imagens que contém comidas. Seu dever é identificar a comida presente em cada imagem e dizer quais comidas estão presentes. Use o seguinte padrão:
    
    [Inicio da lista]
    - "pizza"
    - "hamburguer"
    - "sushi"
    - ...
    [Fim da lista]
    
    """
    
    message_prompt = SystemMessage(
        content=system_prompt
    )
    
    all_content = []
    for i, image_data in enumerate(images_data):
        content = [
            {"type": "text", "text": f"Imagem - {i}:"},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ]
        all_content += content
    
    
    message_img = HumanMessage(
        content=all_content
    )
    
    messages = [message_prompt, 
                message_img]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    
    chain_extraction = prompt | llm | StrOutputParser()
    
    response = chain_extraction.invoke({})
    return response

In [17]:
path_imgs = "../data/imgs_food"
paths = [os.path.join(path_imgs, x) for x in os.listdir(path_imgs)]
response = get_list_food(paths=paths)

In [19]:
print(response)

[Inicio da lista]
- "batata frita"
- "moqueca de camarão"
- "cuscuz"
- "tapioca"
[Fim da lista]


### Multimodalidade + Structured Output

Além do benefício de compreender o conteúdo presente nas imagens, podemos unir essa funcionalidade com a funcionalidade de estruturação de outputs. Isto é, podemos utilizar o modelo para extrair informações de imagens e, em seguida, estruturar essas informações em um formato específico. Isso é de suma importância na construção de APIs que garantem confiabilidade e consistência nos outputs.

In [22]:
model_name = "gpt-4o-2024-08-06"

llm = ChatOpenAI(model=model_name, temperature=0.01)

class FoodSchema(BaseModel):
    """Estrutura todas as comidas presentes no texto/imagem descrito."""
    
    foods: str = Field(description="Comida presente no texto/imagem.", examples=["pizza", 
                                                                                 "hamburguer", 
                                                                                 "sushi", 
                                                                                 "salada"])

def get_structured_list_food(paths: List[str], 
                          provider="openai"):

    # Caminho para a imagem em Base64
    image_urls = paths 
    delta = len(image_urls)
    images_data = []
    
    for image_url in image_urls:
        with open(image_url, "rb") as image_file:
            image_data = base64.b64encode(image_file.read()).decode("utf-8")
            images_data.append(image_data)
    
    system_prompt = """
    
    Você vai receber uma séria de imagens que contém comidas. Seu dever é identificar a comida presente em cada imagem e dizer quais comidas estão presentes. Use o seguinte padrão:
    
    [Inicio da lista]
    - "pizza"
    - "hamburguer"
    - "sushi"
    - ...
    [Fim da lista]
    
    """
    
    message_prompt = SystemMessage(
        content=system_prompt
    )
    
    all_content = []
    for i, image_data in enumerate(images_data):
        content = [
            {"type": "text", "text": f"Imagem - {i}:"},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
            },
        ]
        all_content += content
    
    
    message_img = HumanMessage(
        content=all_content
    )
    
    messages = [message_prompt, 
                message_img]
    
    prompt = ChatPromptTemplate.from_messages(messages)
    
    system_prompt_structured = """
    
    Você vai receber uma lista em texto de comidas registradas de imagens anteriores. Seu dever é identificar a comida presente no texto e estruturar as comidas presentes. 
    
    """
    
    prompt_structured = ChatPromptTemplate.from_messages(
                    [
                        ("system", system_prompt_structured),
                        ("human", "Lista de comida: {input}"),
                    ]
                )
    
    chain_extraction = prompt | llm | StrOutputParser() | {"input": RunnablePassthrough()}
    
    llm_with_tools_extraction = llm.bind_tools([FoodSchema]) 
    
    chain_structured_extraction = prompt_structured | llm_with_tools_extraction
    
    final_chain = chain_extraction | chain_structured_extraction
    
    response = final_chain.invoke({})
    
    return response

In [23]:
path_imgs = "../data/imgs_food"
paths = [os.path.join(path_imgs, x) for x in os.listdir(path_imgs)]
response = get_structured_list_food(paths=paths)

In [25]:
response.tool_calls

[{'name': 'FoodSchema',
  'args': {'foods': 'batata frita'},
  'id': 'call_eZ8u6ObbNJMKiqUMeib6I2PS',
  'type': 'tool_call'},
 {'name': 'FoodSchema',
  'args': {'foods': 'moqueca de camarão'},
  'id': 'call_hDnSyy3LiW0dPFEy1dCB8b8x',
  'type': 'tool_call'},
 {'name': 'FoodSchema',
  'args': {'foods': 'cuscuz'},
  'id': 'call_ezxlDcP91aOfoyGuqfyh8POv',
  'type': 'tool_call'},
 {'name': 'FoodSchema',
  'args': {'foods': 'tapioca'},
  'id': 'call_9M3c0plNsxZuO3kOXkFipxIB',
  'type': 'tool_call'}]